Copy the SVD files produced by the preprocessing program to the new directory
svd-cnn

In [2]:
# first change the directory
import os

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)


path = "D:\CSCI8363-Fall2017\project idea\svd-cnn"
createFolder(path)
os.chdir(path)
cwd = os.getcwd()
print(cwd)

D:\CSCI8363-Fall2017\project idea\svd-cnn


In [3]:
import csv
import numpy as np
U0 = np.loadtxt(open("SVD_U0.csv", "rb"), delimiter=",", skiprows=0)
U1 = np.loadtxt(open("SVD_U1.csv", "rb"), delimiter=",", skiprows=0)
U2 = np.loadtxt(open("SVD_U2.csv", "rb"), delimiter=",", skiprows=0)
U3 = np.loadtxt(open("SVD_U3.csv", "rb"), delimiter=",", skiprows=0)
U4 = np.loadtxt(open("SVD_U4.csv", "rb"), delimiter=",", skiprows=0)
U5 = np.loadtxt(open("SVD_U5.csv", "rb"), delimiter=",", skiprows=0)
U6 = np.loadtxt(open("SVD_U6.csv", "rb"), delimiter=",", skiprows=0)
U7 = np.loadtxt(open("SVD_U7.csv", "rb"), delimiter=",", skiprows=0)
U8 = np.loadtxt(open("SVD_U8.csv", "rb"), delimiter=",", skiprows=0)
U9 = np.loadtxt(open("SVD_U9.csv", "rb"), delimiter=",", skiprows=0)

In [4]:
U9.shape

(784L, 784L)

Reduce the dimension of the U matrix and save the only leading 10 columns

In [5]:
r = 10
U0_reduce = U0[:,0:r]
U0_new = np.dot(U0_reduce,np.transpose(U0_reduce))
U1_reduce = U1[:,0:r]
U1_new = np.dot(U1_reduce,np.transpose(U1_reduce))
U2_reduce = U2[:,0:r]
U2_new = np.dot(U2_reduce,np.transpose(U2_reduce))
U3_reduce = U3[:,0:r]
U3_new = np.dot(U3_reduce,np.transpose(U3_reduce))
U4_reduce = U4[:,0:r]
U4_new = np.dot(U4_reduce,np.transpose(U4_reduce))
U5_reduce = U5[:,0:r]
U5_new = np.dot(U5_reduce,np.transpose(U5_reduce))
U6_reduce = U6[:,0:r]
U6_new = np.dot(U6_reduce,np.transpose(U6_reduce))
U7_reduce = U7[:,0:r]
U7_new = np.dot(U7_reduce,np.transpose(U7_reduce))
U8_reduce = U8[:,0:r]
U8_new = np.dot(U8_reduce,np.transpose(U8_reduce))
U9_reduce = U9[:,0:r]
U9_new = np.dot(U9_reduce,np.transpose(U9_reduce))
U_assembly = np.zeros((784,784,10)) # Make a 10 by 20 by 30 array
U_assembly[:,:,0] = U0_new
U_assembly[:,:,1] = U1_new
U_assembly[:,:,2] = U2_new
U_assembly[:,:,3] = U3_new
U_assembly[:,:,4] = U4_new
U_assembly[:,:,5] = U5_new
U_assembly[:,:,6] = U6_new
U_assembly[:,:,7] = U7_new
U_assembly[:,:,8] = U8_new
U_assembly[:,:,9] = U9_new

U_reduce_assembly = np.zeros((784,10,10)) # Make a 10 by 20 by 30 array
U_reduce_assembly[:,:,0] = U0_reduce
U_reduce_assembly[:,:,1] = U1_reduce
U_reduce_assembly[:,:,2] = U2_reduce
U_reduce_assembly[:,:,3] = U3_reduce
U_reduce_assembly[:,:,4] = U4_reduce
U_reduce_assembly[:,:,5] = U5_reduce
U_reduce_assembly[:,:,6] = U6_reduce
U_reduce_assembly[:,:,7] = U7_reduce
U_reduce_assembly[:,:,8] = U8_reduce
U_reduce_assembly[:,:,9] = U9_reduce
U_reduce_assembly_total = np.zeros((784,100)) # Make a 10 by 20 by 30 array
U_reduce_assembly_total = np.concatenate((U0_reduce, U1_reduce, U2_reduce, U3_reduce, U4_reduce,\
                           U5_reduce, U6_reduce, U7_reduce, U8_reduce, U9_reduce), axis=1)

In [26]:
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

In [36]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train.shape
print(y_train.shape)
U_reduce_assembly= np.asarray(U_reduce_assembly)
U_reduce_assembly.shape

(60000,)


(784, 10, 10)

In [37]:
X_train_coefficient = np.zeros((X_train.shape[0],10))
for i in range(X_train.shape[0]):
    a = U_reduce_assembly[:,:,y_train[i]]
    b = np.linalg.lstsq(a, X_train[i,:,:].reshape(784))
    X_train_coefficient[i,:]= b[0]

In [39]:
X_test_coefficient = np.zeros((X_test.shape[0],10))
for i in range(X_test.shape[0]):
    atest= U_reduce_assembly[:,:,y_test[i]]
    btest = np.linalg.lstsq(atest, X_test[i,:,:].reshape(784))
    X_test_coefficient[i,:] = b[0]

In [40]:
# reshape to be [samples][pixels][width][height]
X_train_new = X_train_coefficient.reshape(X_train_coefficient.shape[0], 1, 1 , 10).astype('float32')
X_test_new = X_test_coefficient.reshape(X_test_coefficient.shape[0], 1, 1, 10).astype('float32')

In [41]:
# one hot encode outputs, change to binary matrix
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [45]:
def Simple_CNN_model():
    # create model
    model = Sequential()
    model.add(Conv2D(32, (1, 1), input_shape=(1, 1, 10), activation='relu'))
    model.add(MaxPooling2D(pool_size=(1, 1)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(20, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [46]:
# build the model
model = Simple_CNN_model()
# Fit the model
model.fit(X_train_new, y_train, epochs=10, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test_new, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))
#scores1 = model.evaluate(X_test_noisy, y_test_noisy, verbose=0)
#print("CNN Error noisy data: %.2f%%" % (100-scores1[1]*100))

Epoch 1/10
 - 5s - loss: 13.9850 - acc: 0.1215
Epoch 2/10
 - 5s - loss: 13.6380 - acc: 0.1436
Epoch 3/10
 - 5s - loss: 13.4353 - acc: 0.1546
Epoch 4/10
 - 5s - loss: 13.2755 - acc: 0.1638
Epoch 5/10
 - 5s - loss: 11.8427 - acc: 0.1598
Epoch 6/10
 - 5s - loss: 2.3676 - acc: 0.1124
Epoch 7/10
 - 5s - loss: 2.3126 - acc: 0.1124
Epoch 8/10
 - 5s - loss: 2.3063 - acc: 0.1124
Epoch 9/10
 - 5s - loss: 2.3039 - acc: 0.1124
Epoch 10/10
 - 5s - loss: 2.3025 - acc: 0.1125
CNN Error: 88.65%
